# Hello and welcome to the notebook
## ANother heading
### A Third HEading

In [1]:
import openmc
import openmc.deplete
from simple_openmc_depletion import create_openmc_model
from extract_simple_depletion_results import plot_results

# Step 1: Setup the Tranpsort Operator